# PETs/TETs – Hyperledger Aries – Authority Agent (Issuing Authority) 🏛️


In [1]:
%%javascript
document.title='🏛️ Authority'

<IPython.core.display.Javascript object>

## PART 2: Issue a VC to the Manufacturer Agents

**What:** Issue verifiable credentials (VCs) to all manufacturers

**Why:** Manufacturers will be able to store VCs, and prove to the city (the data scientist) that they are manufacturers, without revealing their identity.

**How:** <br>
1. [Initiate Authority's AgentCommunicationManager (ACM)](#1) <br>
2. [Connect with Manufacturer1](#2)
3. [Issue VC to Manufacturer1](#3)
4. [Repeat Steps 2-3 for Manufacturer2](#4)
4. [Repeat Steps 2-3 for Manufacturer3](#5)

**Accompanying Agents and Notebooks:** 
* Manufacturer1 🚗: `02_get_manufacturer1_VC.ipynb`
* Manufacturer2 🚛: `02_get_manufacturer2_VC.ipynb`
* Manufacturer3 🛵: `02_get_manufacturer3_VC.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import os

from aries_cloudcontroller import AriesAgentController

import libs.helpers as helpers
from libs.agent_connection_manager import IssuingAuthority

#### 0.2 – Variables

In [3]:
# Get identifier data defined in notebook 00_init_authority_as_issuingAuthority.ipynb
identifiers = helpers.get_identifiers()
schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]
cred_def_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["cred_def"]

# Get environment variables
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

Get stored identifiers dictionary to access schema information 💾


---

<a id=1> </a>
### 1 – Initiate Authority Agent

#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://authority-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM issuing authority

In [6]:
# The IssuingAuthority registers relevant webhook servers and event listeners
authority_agent = IssuingAuthority(agent_controller)

Successfully initiated AgentConnectionManager for a(n) Issuing Authority ACA-PY agent


---

<a id=2> </a>

### 2 – Establish a connection with Manufacturer1 🚗
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the manufacturer1 requests a connection with the Authority to be certified as an official city agency. Thus, the manufacturer1 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call 💻, phone ☎️, E-Mail 📧, or fax 📠. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

### 2.1 – Receive invitation from `Manufacturer1` agent
Copy the invitation from Step 2.1 in the City's `01_hold_VC.ipynb` notebook into the following cell.
Several state changes of the connection between the Manufacturer agent, the inviter (A), and the authority agent, the invitee (B),  are required before successfully establishing a connection: 

| Step | State | Agent | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | invitation-sent | A | A sent an invitation to B | `create_connection_invitation()`
| 2 | invitation-received | B | B receives the invitation of A | Prompt: Paste invitation from A |
| 3 | request-sent | B | B sends B connection request | Prompt: Accept invitation OR `auto_accept=True` |
| 4 | request-received | A | A receives the connection request from B | Prompt: Accept invitation request response OR `auto_accept=True` |
| 5 | response-sent | A | A sends B response to B | - |
| 6 | response-received | B | B receives the response from A | - |
| 7 | active (completed) | A | B pings A to finalize connection | Prompt: Trust ping OR `auto_ping=True` |


In [8]:
# Variables
alias = None
auto_accept= True

# Receive connection invitation
connection_id_m1 = authority_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

Please enter invitation received by external agent:


Invitation:  {     '@id': 'a48b8be7-2003-4d18-9b7a-8524f7d3ebcb',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'AnonymousAgent1',     'recipientKeys': ['66MmatAydAqRyNGAgFFzuv8V43NdzKcAiyaXkBRCksBL'],     'serviceEndpoint': 'https://fab5-80-134-223-153.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9fee067d-e637-42d7-b4fa-936209ba4ae9
State :  invitation (invitation-received)
Routing State : none
Connection with :  AnonymousAgent1
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9fee067d-e637-42d7-b4fa-936209ba4ae9
State :  request (request-sent)
Routing State : none
Connection with :  AnonymousAgent1
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9fee067d-e637-42d7-b4fa-936209ba4ae9
State :  response (response-received)
Routing State : none
Connection with :  AnonymousAge

<div style="font-size: 25px"><center><b>Break Point 2</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🚗</center></div><br>
<center><b>Please return to the Manufacturer1's notebook 🚗. <br>Check the prompts in Step 2.1 (e.g., if auto_accept or auto_ping are set to False), and then proceed to Step 3.</b></center>

---

<a id=3> </a>
## 3 – Process VC request by Manufacturers
### 3.1 – Check messages / requests by Manufacturers
Check inbox and await messages sent by Manufacturer1 🚗

In [9]:
# Verify inbox
message_ids = authority_agent.verify_inbox()


---------------------------------------------------------------------
Message Inbox
> 2 Message(s) via Connection ID 9fee067d-e637-42d7-b4fa-936209ba4ae9:
	 * Message ID :  276b4c6d-8acc-45f7-9da4-d0f8beeec24e
	 * Message ID :  b2a66b10-8127-4d07-a2a6-d371e57917ba
---------------------------------------------------------------------


In [10]:
for m_id in message_ids:
    authority_agent.get_message(m_id)


---------------------------------------------------------------------
Message received
Connection ID :  9fee067d-e637-42d7-b4fa-936209ba4ae9
Message ID :  276b4c6d-8acc-45f7-9da4-d0f8beeec24e
State :  received
Time :  2021-08-24 11:43:17.124354Z
Text :  Hello Authority. I would like to be certified as a manufacturer.
---------------------------------------------------------------------

---------------------------------------------------------------------
Message received
Connection ID :  9fee067d-e637-42d7-b4fa-936209ba4ae9
Message ID :  b2a66b10-8127-4d07-a2a6-d371e57917ba
State :  received
Time :  2021-08-24 11:43:17.160710Z
Text :  {"manufacturerName": "undisclosedName1", "manufacturerCountry": "DE", "manufacturerCity": "Berlin"}
---------------------------------------------------------------------


### 3.3 – Offer VC to `Manufacturer1` agent 🚗
The next step is to offer a VC to the manufacturer agent. The manufacturer can then request the offer and store it in their wallet. The following table provides an overview of the individual states between I (Issuer, the Authority agent) and H (Holder, the Manufacturer).

| Step | State | Role | Description | Function/Prompt/Variable |
| --- | --- | --- | --- | --- |
| 1 | offer_sent | I | I sends I VC offer with personalized information to H| `offer_vc()` |
| 2 | offer_received | H | H receives offer made by I | - |
| 3 | request_sent | H | Request VC offer | `request_vc()` AND (Prompt: request VC OR `auto_request=True`) |
| 4 | request_received | I | M1's request to get VC was received | - |
| 5 | credential_issued | I | Automatic response to issue credential | - |
| 6 | credential_received | H| H receives VC and is asked to store it | Prompt: Store VC OR `auto_store=True`
| 7 | credential_acked | I / H | Credential was issued and stored | - |

If you enter the information that was sent by the Manufacturer1 Agent (see `Text` attribute in message) when prompted, the proposed credential should look something like this: 
```
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'manufacturerCity', 'value': 'Berlin'},
        {'name': 'manufacturerName', 'value': 'Manufacturer1'},
        {'name': 'manufacturerCountry', 'value': 'Germany'},
        {'name': 'isManufacturer', 'value': 'TRUE'},
    ],
}
```

In [11]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer1 is a manufacturer"
auto_remove = True
trace = False

# Offer Manufacturer1 a VC with manufacturer_schema
authority_agent.offer_vc(
    connection_id_m1, 
    schema_manufacturer_id, 
    cred_def_manufacturer_id, 
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    #credential_attributes=[{"name": "manufacturerName", "value": "undisclosedManufacturer1"}, {"name": "manufacturerCity", "value": "Berlin"}, {"name": "manufacturerCountry", "value": "Germany"}, {"name": "isManufacturer", "value": "TRUE"}]
)

Please enter the following information for the certify-manufacturer scheme: 


isManufacturer:  TRUE
manufacturerCountry:  DE
manufacturerName:  carManufacturer
manufacturerCity:  City1
Is the information correct? [yes/no] yes



---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : 9fee067d-e637-42d7-b4fa-936209ba4ae9
Credential exchange ID : 3424d747-12e0-480a-9cbf-43bad78b09c7
Agent Protocol Role :  issuer
Protocol State :  offer_sent
---------------------------------------------------------------------

Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'isManufacturer', 'value': 'TRUE'},
        {'name': 'manufacturerCountry', 'value': 'DE'},
        {'name': 'manufacturerName', 'value': 'carManufacturer'},
        {'name': 'manufacturerCity', 'value': 'City1'},
    ],
}

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : 9fee067d-e637-42d7-b4fa-936209ba4ae9
Credential exchange ID : 3424d747-12e0-480a-9cbf-43bad78b09c7
Agent

<div style="font-size: 25px"><center><b>Break Point 4</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🚗</center></div><br>
<center><b>Please return to the Manufacturer1's notebook 🚗. <br>Continue with Step 3.2</b></center>

---

<a id=4> </a>
### 4 – 🔁 Repeat Steps 2 and 3 with Manufacturer2 🚛
🤦 Execute the following cells to certify Manufacturer2 🚛 that the agent is a manufacturer.
#### 4.1 – Receive connection invitation by Manufacturer2

In [13]:
# Variables
alias = None
auto_accept= True

# Receive connection invitation
connection_id_m2= authority_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

Please enter invitation received by external agent:


Invitation:  {     '@id': 'ba96b808-778f-47c5-912f-54503946929f',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'AnonymousAgent2',     'recipientKeys': ['9AUYUUcBouJm8zvsk3oHxstnHEx7uynNPiBvP4U3Rvde'],     'serviceEndpoint': 'https://7ae3-80-134-223-153.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  a1ad5635-3f46-4a93-8bcc-d67a186a6363
State :  invitation (invitation-received)
Routing State : none
Connection with :  AnonymousAgent2
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  a1ad5635-3f46-4a93-8bcc-d67a186a6363
State :  request (request-sent)
Routing State : none
Connection with :  AnonymousAgent2
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  a1ad5635-3f46-4a93-8bcc-d67a186a6363
State :  response (response-received)
Routing State : none
Connection with :  AnonymousAge

#### 4.2 Offer VC to Manufacturer2

In [14]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer2 is a manufacturer"
auto_remove = True
trace = False

# Offer Manufacturer1 a VC with manufacturer_schema
authority_agent.offer_vc(
    connection_id_m2, 
    schema_manufacturer_id, 
    cred_def_manufacturer_id, 
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    credential_attributes=[{"name": "manufacturerName", "value": "truckManufacturer"}, {"name": "manufacturerCity", "value": "City2"}, {"name": "manufacturerCountry", "value": "DE"}, {"name": "isManufacturer", "value": "TRUE"}]
)


---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : a1ad5635-3f46-4a93-8bcc-d67a186a6363
Credential exchange ID : 97086e27-4cf7-4611-8b4d-c0ab0a15b865
Agent Protocol Role :  issuer
Protocol State :  offer_sent
---------------------------------------------------------------------

Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'manufacturerName', 'value': 'truckManufacturer'},
        {'name': 'manufacturerCity', 'value': 'City2'},
        {'name': 'manufacturerCountry', 'value': 'DE'},
        {'name': 'isManufacturer', 'value': 'TRUE'},
    ],
}

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : a1ad5635-3f46-4a93-8bcc-d67a186a6363
Credential exchange ID : 97086e27-4cf7-4611-8b4d-c0ab0a15b865
Age

<div style="font-size: 25px"><center><b>Break Point 6</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🚛</center></div><br>
<center><b>Please return to the Manufacturer2's notebook 🚛. <br>Continue with Step 3.1</b></center>

---

<a id=5> </a>
### 5 – 🔁 Repeat Steps 2 and 3 with Manufacturer3 🛵
🙇 Execute the following cells to certify Manufacturer3 that the agent is a manufacturer.
#### 5.1 – Establish a connection with Manufacturer3
All variables are set to auto_accept to speed up the conneciton process.

In [15]:
# Variables
alias = None
auto_accept= True

# Receive connection invitation
connection_id_m3 = authority_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

Please enter invitation received by external agent:


Invitation:  {     '@id': '353b75ac-9ed7-4141-90b0-c20716363b68',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'AnonymousAgent3',     'recipientKeys': ['9ey4JoFjvYqHpZEh3x53XoytPmrdXoDAaNq6xtWL7676'],     'serviceEndpoint': 'https://3770-80-134-223-153.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  ba1aad91-e8cf-43c2-897f-ced025cbdeb9
State :  invitation (invitation-received)
Routing State : none
Connection with :  AnonymousAgent3
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  ba1aad91-e8cf-43c2-897f-ced025cbdeb9
State :  request (request-sent)
Routing State : none
Connection with :  AnonymousAgent3
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  ba1aad91-e8cf-43c2-897f-ced025cbdeb9
State :  response (response-received)
Routing State : none
Connection with :  AnonymousAge

#### 5.2 Offer VC to Manufacturer2

In [16]:
# MAKE VC ZKP-able! SEE https://github.com/hyperledger/aries-cloudagent-python/blob/main/JsonLdCredentials.md
comment = "Issuing VC that Manufacturer3 is a manufacturer"
auto_remove = True
trace = False

# Offer Manufacturer1 a VC with manufacturer_schema
authority_agent.offer_vc(
    connection_id_m3, 
    schema_manufacturer_id, 
    cred_def_manufacturer_id, 
    comment=comment, 
    # Comment out next line if you do not want to get the prompts to enter VC information
    credential_attributes=[{"name": "manufacturerName", "value": "scooterManufacturer"}, {"name": "manufacturerCity", "value": "City3"}, {"name": "manufacturerCountry", "value": "DE"}, {"name": "isManufacturer", "value": "TRUE"}]
)


---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : ba1aad91-e8cf-43c2-897f-ced025cbdeb9
Credential exchange ID : 3f75c3ec-e65d-41bc-9354-a10d395abfa6
Agent Protocol Role :  issuer
Protocol State :  offer_sent
---------------------------------------------------------------------

Proposed Credential : 
{
    '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/issue-credential/1.0/credential-preview',
    'attributes': [
        {'name': 'manufacturerName', 'value': 'scooterManufacturer'},
        {'name': 'manufacturerCity', 'value': 'City3'},
        {'name': 'manufacturerCountry', 'value': 'DE'},
        {'name': 'isManufacturer', 'value': 'TRUE'},
    ],
}

---------------------------------------------------------------------
Handle Issue Credential Webhook: Issue Credential Handler
Connection ID : ba1aad91-e8cf-43c2-897f-ced025cbdeb9
Credential exchange ID : 3f75c3ec-e65d-41bc-9354-a10d395abfa6
A

<div style="font-size: 25px"><center><b>Break Point 9</b></center></div>
<div style="font-size: 50px"><center>🏛 ➡️ 🛵</center></div><br>
<center><b>Please return to the Manufacturer3's notebook 🛵. <br>Continue with Step 3.1</b></center>

---

## 6 - Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [17]:
await agent_controller.terminate()

--- 

### 🔥🔥🔥 You are done 🙌 and can close this notebook now 🔥🔥🔥
